# 데이터셋 다운로드

### https://drive.google.com/drive/folders/1z-FSY3JN-FWBl87XAZyb4m2KcHqJEUTm?usp=sharing

### Colab 사용 시 Colab repository에 업로드 할 것.

In [ ]:
from tensorflow.keras import Model
from tensorflow.keras.applications import MobileNetV2
import tensorflow.keras.layers as layers
import tensorflow as tf
import numpy as np
import os
import cv2
from matplotlib import pyplot as plt

# A. Backbone 신경망의 전이학습

In [ ]:
mobilenet_v2 = MobileNetV2(input_shape=(224,224,3), weights="imagenet", include_top=False)
tf.keras.utils.plot_model(mobilenet_v2, show_shapes=True, expand_nested=True)

start = mobilenet_v2.get_layer("input_1").input
end = mobilenet_v2.get_layer("block_5_add").output
backbone = tf.keras.Model(start, end)

# B. Neck 신경망 연결

In [ ]:
#input shape (BN, H, W, C)

block1 = tf.keras.Sequential([tf.keras.layers.Conv2D(256, 1, padding='same'),
                     tf.keras.layers.BatchNormalization(),
                     tf.keras.layers.ReLU(),

                     tf.keras.layers.DepthwiseConv2D(3, strides=(2,2), padding='same'),
                     tf.keras.layers.BatchNormalization(),
                     tf.keras.layers.ReLU(),

                     tf.keras.layers.Conv2D(256, 1, padding='same'),
                     tf.keras.layers.BatchNormalization(),
                     tf.keras.layers.ReLU()])
                    # **** conv => depthwise-conv => conv ****
                    #output1 [BN,H/2,W/2,512]

block2 = tf.keras.Sequential([tf.keras.layers.Conv2D(256, 1, padding='same'),
                    tf.keras.layers.BatchNormalization(),
                    tf.keras.layers.ReLU(),

                    tf.keras.layers.DepthwiseConv2D(3, strides=(2,2), padding='same'),
                    tf.keras.layers.BatchNormalization(),
                    tf.keras.layers.ReLU(),

                    tf.keras.layers.Conv2D(256, 1, padding='same'),
                    tf.keras.layers.BatchNormalization(),
                    tf.keras.layers.ReLU(),])
                    # **** conv => depthwise-conv => conv ****
                    #output1 [BN,H/4,W/4,512]
        
block3 = tf.keras.Sequential([tf.keras.layers.Conv2D(256, 1, padding='same'),
                    tf.keras.layers.BatchNormalization(),
                    tf.keras.layers.ReLU(),

                    tf.keras.layers.DepthwiseConv2D(3, strides=(2,2)),
                    tf.keras.layers.BatchNormalization(),
                    tf.keras.layers.ReLU(),

                    tf.keras.layers.Conv2D(256, 1, padding='same'),
                    tf.keras.layers.BatchNormalization(),
                    tf.keras.layers.ReLU()])
                    # **** conv => depthwise-conv => conv ****
                    #output1 [BN,H/8,W/8,512]
        
block4 = tf.keras.Sequential([tf.keras.layers.Conv2D(256, 1, padding='same'),
                    tf.keras.layers.BatchNormalization(),
                    tf.keras.layers.ReLU(),

                    tf.keras.layers.DepthwiseConv2D(3, strides=(2,2)),
                    tf.keras.layers.BatchNormalization(),
                    tf.keras.layers.ReLU(),

                    tf.keras.layers.Conv2D(256, 1, padding='same'),
                    tf.keras.layers.BatchNormalization(),
                    tf.keras.layers.ReLU()])
                    # **** conv => depthwise-conv => conv ****
                    #output1 [BN,H/16,W/16,512]

In [ ]:
input_layer = tf.keras.Input(shape=(224,224,3))
x = backbone(input_layer)
y1 = block1(x)
y2 = block2(y1)
y3 = block3(y2)
y4 = block4(y3)

print("첫 번째 feature map : ", x.shape,\
      "두 번째 feature map : ", y1.shape,\
      "세 번째 feature map : ", y2.shape,\
      "네 번쨰 feature map : ",y3.shape,\
      "다섯번째 feature map : ",y4.shape)

# C. head(classification, regression) 신경망 연결

In [ ]:
num_classes = 3

#classification head
confs = []
conf = layers.Conv2D(4 * num_classes, kernel_size=3, padding='same')(x)
conf = tf.reshape(conf, [-1, 28*28*4,num_classes])
confs.append(conf)
conf = layers.Conv2D(6 * num_classes, kernel_size=3, padding='same')(y1)
conf = tf.reshape(conf, [-1, 14*14*6,num_classes])
confs.append(conf)
conf = layers.Conv2D(6 * num_classes, kernel_size=3, padding='same')(y2)
conf = tf.reshape(conf, [-1, 7*7*6,num_classes])
confs.append(conf)
conf = layers.Conv2D(6 * num_classes, kernel_size=3, padding='same')(y3)
conf = tf.reshape(conf, [-1, 3*3*6,num_classes])
confs.append(conf)
conf = layers.Conv2D(4 * num_classes, kernel_size=1)(y4)
conf = tf.reshape(conf, [-1, 1*1*4,num_classes])
confs.append(conf)


#regression head
locs = []
loc = layers.Conv2D(4 * 4, kernel_size=3, padding='same')(x)
loc = tf.reshape(loc, [-1, 28*28*4,4])
print(loc.shape)
locs.append(loc)
loc = layers.Conv2D(6 * 4, kernel_size=3, padding='same')(y1)
loc = tf.reshape(loc, [-1, 14*14*6,4])
print(loc.shape)
locs.append(loc)
loc = layers.Conv2D(6 * 4, kernel_size=3, padding='same')(y2)
loc = tf.reshape(loc, [-1, 7*7*6,4])
print(loc.shape)
locs.append(loc)
loc = layers.Conv2D(6 * 4, kernel_size=3, padding='same')(y3)
loc = tf.reshape(loc, [-1, 3*3*6,4])
print(loc.shape)
locs.append(loc)
loc = layers.Conv2D(4 * 4, kernel_size=1)(y4)
loc = tf.reshape(loc, [-1, 1*1*4,4])
print(loc.shape)
locs.append(loc)

confs = tf.concat(confs, axis=-2)
locs = tf.concat(locs, axis=-2)

# D. 최종 모델 빌드

In [ ]:
mobilenet_ssd = tf.keras.Model(inputs=[input_layer], outputs=[confs,locs])

In [ ]:
tf.keras.utils.plot_model(mobilenet_ssd, show_shapes=True, expand_nested=False)

In [ ]:
mobilenet_ssd(np.ones([1, 224, 224, 3], np.float32))

# SSD 유틸러티 모듈 다운로드 & 데이터셋 다운로드

In [ ]:
!git clone https://github.com/hukim1112/MLDL.git
import sys
sys.path.append("/content/MLDL/object_detection")

In [ ]:
#데이터셋 업로드 후 압축 풀 것.
!unzip images.zip

In [ ]:
image_dir = '/content/images'
annotation = '/content/GDUT_HWD.json'

#압축
img_list = os.listdir(image_dir)
img_path = os.path.join(image_dir, img_list[0])
img = cv2.imread(img_path)[:,:,::-1]
plt.imshow(img)

# 학습 configuration

In [ ]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
import anchor, losses, manage_checkpoint, coco, post_process
from pycocotools.coco import COCO

In [ ]:
config = {
    "label_set" : ["head","helmet"], # 분류할 오브젝트 집합
    "input_shape" : [224, 224, 3], #모델입력영상 크기
    "anchor_param" : {"ratios": [[2], [2, 3], [2, 3], [2, 3], [2]],
                           "scales": [0.1, 0.3, 0.5, 0.7, 0.9, 1.075],
                           "fm_sizes": [28, 14, 7, 3, 1],
                           "image_size": 224}, #anchor parameters
    "train" :
    {
        "num_examples" : -1,
        "batch_size" : 8,
        "neg_ratio" : 3,
        "initial_lr" : 1e-3,
        "momentum" : 0.9,
        "weight_decay" : 5e-5,
    },
    "val":
    {
        "num_examples" : -1,
        "batch_size" : 8
    },
    "test":
    {
        "num_examples" : -1,
        "batch_size" : 1
    }

}

# 데이터셋 오브젝트 생성

In [ ]:
ds_obj = coco.Dataset(image_dir, annotation, config, COCO)

In [ ]:
train_ds, train_length = ds_obj.load_data_generator('train')
print("Dataset length : ", train_length)

In [ ]:
for i, (_, imgs, gt_confs, gt_locs) in enumerate(train_ds.take(1)):
    print(imgs.shape, gt_confs.shape, gt_locs.shape)

In [ ]:
batch_size = config["train"]["batch_size"]
shuffle_buffer = 159

train_ds = train_ds.shuffle(shuffle_buffer).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
for i, (_, imgs, gt_confs, gt_locs) in enumerate(train_ds.take(1)):
    print(imgs.shape, gt_confs.shape, gt_locs.shape)

# Anchor boxes

In [ ]:
default_boxes = anchor.generate_default_boxes(config["anchor_param"])

In [ ]:
default_boxes.shape

# Train a mobilenet-SSD

In [ ]:
@tf.function
def train_step(imgs, gt_confs, gt_locs, model, criterion, optimizer, config):
    with tf.GradientTape() as tape:
        confs, locs = model(imgs)

        conf_loss, loc_loss = criterion(
            confs, locs, gt_confs, gt_locs)

        loss = conf_loss + loc_loss
        l2_loss = [tf.nn.l2_loss(t) for t in model.trainable_variables]
        l2_loss = config['train']['weight_decay'] * tf.math.reduce_sum(l2_loss)
        loss += l2_loss

    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    return loss, conf_loss, loc_loss, l2_loss

criterion = losses.create_losses(config['train']['neg_ratio'], num_classes)

In [ ]:
t_loss = tf.metrics.Mean(name='train_loss')
t_conf_loss = tf.metrics.Mean(name='train_conf_loss')
t_loc_loss = tf.metrics.Mean(name='train_loc_loss')
# v_loss = tf.metrics.Mean(name='val_loss')
# v_conf_loss = tf.metrics.Mean(name='val_conf_loss')
# v_loc_loss = tf.metrics.Mean(name='val_loc_loss')

learning_rate = 1E-3
EPOCH = 50
START = 0
optimizer = tf.keras.optimizers.Adam(learning_rate)
for epoch in range(START+1, START+EPOCH+1):
    for i, (_, imgs, gt_confs, gt_locs) in enumerate(train_ds):
        imgs = preprocess_input(imgs)
        loss, conf_loss, loc_loss, l2_loss = train_step(
            imgs, gt_confs, gt_locs, mobilenet_ssd, criterion, optimizer, config)
        t_loss(loss)
        t_conf_loss(conf_loss)
        t_loc_loss(loc_loss)
        
        print("Epoch {} iteration {} loss : {}".format(epoch, i, t_loss.result()))
    
    '''
    you can add the code of validation test, but we don't have it now.
    for i, (_, imgs, gt_confs, gt_locs) in enumerate(val_generator):
        imgs = preprocess_input(imgs)
        val_confs, val_locs = model(imgs)
        val_conf_loss, val_loc_loss = criterion(val_confs, val_locs, gt_confs, gt_locs)
        v_loss(val_conf_loss+val_loc_loss)
        v_conf_loss(val_conf_loss)
        v_loc_loss(val_loc_loss)
    '''
    print("Epoch:{}, loss:{}".format(epoch, t_loss.result()))
    t_loss.reset_states()
    t_conf_loss.reset_states()
    t_loc_loss.reset_states()
    #v_loss.reset_states()()))
    #v_conf_loss.reset_states()
    #v_loc_loss.reset_states()   

# 간단한 시각화

In [ ]:
img_list = os.listdir(image_dir)
img_path = os.path.join(image_dir, img_list[0])
img = cv2.imread(img_path)[:,:,::-1]
plt.imshow(img)

H,W = img.shape[:2]
resized_img = cv2.resize(img, (224,224)) #resize
processed_img = preprocess_input(resized_img[tf.newaxis]) #정규화


confs, locs = mobilenet_ssd(processed_img) #예측
results = post_process.predict(confs, locs, default_boxes, num_classes, conf_thresh=0.5) 
# 결과에 Non maximum suppression 적용
print(results)

In [ ]:
drawn_img = img.copy()
for box in results[0][0]:
    x1, y1, x2, y2 = box
    x1 = int(x1*W)
    x2 = int(x2*W)
    y1 = int(y1*H)
    y2 = int(y2*H)
    print(x1, y1, x2, y2)
    drawn_img = cv2.rectangle(drawn_img, (x1,y1), (x2,y2), (255,0,0), 3)
plt.imshow(drawn_img)